<a href="https://colab.research.google.com/github/Sa9Naveen/naveen/blob/main/End_sem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz


--2025-11-06 14:17:34--  http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 192.178.142.207, 142.251.121.207, 142.251.108.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|192.178.142.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G   173MB/s    in 16s     

2025-11-06 14:17:50 (149 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]



In [2]:
!mkdir -p /content/speech_commands_v0.02 && tar -xvf /content/speech_commands_v0.02.tar.gz -C /content/speech_commands_v0.02

Streaming output truncated to the last 5000 lines.
./on/fc3ba625_nohash_0.wav
./on/db9d52d2_nohash_0.wav
./on/72242187_nohash_4.wav
./on/b0f5b16d_nohash_0.wav
./on/d9aa8c90_nohash_1.wav
./on/525eaa62_nohash_0.wav
./on/42e3f068_nohash_0.wav
./on/94d370bf_nohash_4.wav
./on/bf8d5617_nohash_0.wav
./on/e2b7d6d6_nohash_2.wav
./on/89ed36ab_nohash_0.wav
./on/30276d03_nohash_1.wav
./on/364c0853_nohash_0.wav
./on/7cbf645a_nohash_0.wav
./on/c98bbbd3_nohash_2.wav
./on/9a7c1f83_nohash_5.wav
./on/3589bc72_nohash_0.wav
./on/3ae5c04f_nohash_0.wav
./on/adc216c5_nohash_2.wav
./on/c50f55b8_nohash_15.wav
./on/324210dd_nohash_2.wav
./on/c120e80e_nohash_2.wav
./on/69086eb0_nohash_0.wav
./on/dce05676_nohash_1.wav
./on/435695e3_nohash_1.wav
./on/179a61b7_nohash_1.wav
./on/f5341341_nohash_0.wav
./on/91b03183_nohash_0.wav
./on/96d8bb6f_nohash_2.wav
./on/6a1908f8_nohash_0.wav
./on/6ace4fe1_nohash_1.wav
./on/84d1e469_nohash_1.wav
./on/ba676390_nohash_0.wav
./on/1dc86f91_nohash_4.wav
./on/3a929277_nohash_2.wav
./o

In [3]:
DATASET_DRIVE_PATH = "/content/speech_commands_v0.02"

In [4]:
import os, re, hashlib, glob
import numpy as np
from tqdm import tqdm
import librosa

MAX_NUM_WAVS_PER_CLASS = 2**27 - 1

def which_set(filename, validation_percentage, testing_percentage):
    base_name = os.path.basename(filename)
    hash_name = re.sub(r'_nohash_.*$', '', base_name)
    hash_name_bytes = hash_name.encode('utf-8')
    hash_value = int(hashlib.sha1(hash_name_bytes).hexdigest(), 16) % MAX_NUM_WAVS_PER_CLASS
    percentage_hash = float(hash_value) / MAX_NUM_WAVS_PER_CLASS * 100.0
    if percentage_hash < validation_percentage:
        return 'validation'
    elif percentage_hash < (validation_percentage + testing_percentage):
        return 'testing'
    else:
        return 'training'


In [5]:
VALIDATION_PERCENT = 10.0
TEST_PERCENT = 10.0

dataset_path = DATASET_DRIVE_PATH  # from step 1
all_wavs = glob.glob(os.path.join(dataset_path, '**', '*.wav'), recursive=True)

# Filter the audio files to include only the specified words
WORDS = ['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']
filtered_wavs = [f for f in all_wavs if os.path.basename(os.path.dirname(f)) in WORDS]

print(f"Found {len(filtered_wavs)} wav files for the specified words")

train_files, val_files, test_files = [], [], []
for f in filtered_wavs:
    setname = which_set(f, VALIDATION_PERCENT, TEST_PERCENT)
    if setname == 'training':
        train_files.append(f)
    elif setname == 'validation':
        val_files.append(f)
    else:
        test_files.append(f)

print('Counts -> train:', len(train_files), 'val:', len(val_files), 'test:', len(test_files))

Found 38546 wav files for the specified words
Counts -> train: 30733 val: 3546 test: 4267


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (DepthwiseConv2D, Conv2D, MaxPooling2D,
                                     Flatten, Dense, Dropout, LeakyReLU, BatchNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import os, re, hashlib, glob # Import necessary libraries
import librosa # Import librosa
import soundfile # Import soundfile
import sklearn # Import scikit-learn
import joblib # Import joblib
import tqdm # Import tqdm

In [ ]:
!pip install scikit-learn tensorflow tqdm joblib

In [7]:
train_files, val_files, test_files = [], [], []
for f in filtered_wavs:
    setname = which_set(f, VALIDATION_PERCENT, TEST_PERCENT)
    if setname == 'training':
        train_files.append(f)
        print(f"Added {f} to training set")
    elif setname == 'validation':
        val_files.append(f)
        print(f"Added {f} to validation set")
    else:
        test_files.append(f)
        print(f"Added {f} to test set")


Streaming output truncated to the last 5000 lines.
Added /content/speech_commands_v0.02/stop/dcc012ec_nohash_0.wav to training set
Added /content/speech_commands_v0.02/stop/42a99aec_nohash_4.wav to training set
Added /content/speech_commands_v0.02/stop/a827e3a1_nohash_2.wav to training set
Added /content/speech_commands_v0.02/stop/9a2e5b3c_nohash_1.wav to training set
Added /content/speech_commands_v0.02/stop/c634a189_nohash_1.wav to training set
Added /content/speech_commands_v0.02/stop/ea0cf37f_nohash_1.wav to validation set
Added /content/speech_commands_v0.02/stop/c24d96eb_nohash_0.wav to training set
Added /content/speech_commands_v0.02/stop/88f3719f_nohash_1.wav to training set
Added /content/speech_commands_v0.02/stop/07ad9b59_nohash_2.wav to training set
Added /content/speech_commands_v0.02/stop/37d38e44_nohash_0.wav to training set
Added /content/speech_commands_v0.02/stop/893705bb_nohash_3.wav to test set
Added /content/speech_commands_v0.02/stop/6904f0cb_nohash_1.wav to trai

In [ ]:
# Define labels based on file paths used for mel spectrogram extraction
y_train_seq = np.array([os.path.basename(os.path.dirname(f)) for f in train_files])
y_val_seq = np.array([os.path.basename(os.path.dirname(f)) for f in val_files])
y_test_seq = np.array([os.path.basename(os.path.dirname(f)) for f in test_files])


In [ ]:
# Function to extract mel spectrogram features
def extract_mel_spec(filepath, n_mels=40, max_len=40):
    y, sr = librosa.load(filepath, sr=None)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if mel_db.shape[1] < max_len:
        pad_width = max_len - mel_db.shape[1]
        mel_db = np.pad(mel_db, ((0,0),(0,pad_width)), mode='constant')
    else:
        mel_db = mel_db[:, :max_len]
    return mel_db

In [ ]:
# Build dataset arrays
X_train_seq = np.array([extract_mel_spec(f) for f in train_files])[..., np.newaxis]
X_val_seq   = np.array([extract_mel_spec(f) for f in val_files])[..., np.newaxis]
X_test_seq  = np.array([extract_mel_spec(f) for f in test_files])[..., np.newaxis]
print(X_train_seq.shape, X_val_seq.shape, X_test_seq.shape)

(30733, 40, 40, 1) (3546, 40, 40, 1) (4267, 40, 40, 1)


In [ ]:
# Encode labels to integers
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train_seq)
y_val_enc = label_encoder.transform(y_val_seq)
y_test_enc = label_encoder.transform(y_test_seq)

num_classes = len(np.unique(y_train_enc))
y_train_cat = tf.keras.utils.to_categorical(y_train_enc, num_classes)
y_val_cat = tf.keras.utils.to_categorical(y_val_enc, num_classes)
y_test_cat = tf.keras.utils.to_categorical(y_test_enc, num_classes)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
model = Sequential([
    # First layer: Depthwise Separable Conv (DS-CNN Block)
    Conv2D(64, (3,3), input_shape=(X_train_seq.shape[1], X_train_seq.shape[2], 1), padding='same'),
    BatchNormalization(),
    LeakyReLU(),

    DepthwiseConv2D((3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(),
    MaxPooling2D((2,2)),
    Dropout(0.2),

    Conv2D(128, (1,1), padding='same'), # Pointwise convolution
    BatchNormalization(),
    LeakyReLU(),

    DepthwiseConv2D((3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(),
    MaxPooling2D((2,2)),
    Dropout(0.2),

    Conv2D(256, (1,1), padding='same'), # Pointwise convolution
    BatchNormalization(),
    LeakyReLU(),

    DepthwiseConv2D((3,3), padding='same'),
    BatchNormalization(),
    LeakyReLU(),
    MaxPooling2D((2,2)),
    Dropout(0.2),

    Flatten(),
    Dense(256),
    LeakyReLU(),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

earlystop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_seq, y_train_cat,
    validation_data=(X_val_seq, y_val_cat),
    epochs=15,
    batch_size=96,
    callbacks=[earlystop]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_seq, y_test_cat)
print(f'Test accuracy: {test_acc:.4f}')

# Plot training history
plt.figure(figsize=(10, 4))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('DS-CNN Training Performance')
plt.show()

# Predict on test set
y_pred_cat = model.predict(X_test_seq)
y_pred_enc = np.argmax(y_pred_cat, axis=1)
y_test_labels = label_encoder.inverse_transform(y_test_enc)
y_pred_labels = label_encoder.inverse_transform(y_pred_enc)

# Print classification report for per-command accuracy
print("\nClassification Report (Per-Command Accuracy):")
print(classification_report(y_test_labels, y_pred_labels, target_names=label_encoder.classes_))

# Calculate and plot confusion matrix
cm = confusion_matrix(y_test_labels, y_pred_labels, labels=label_encoder.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title('Confusion Matrix (DS-CNN Model)')
plt.show()

# Print the number of files executed
print(f"\nNumber of files executed for testing: {len(test_files)}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 433s 1s/step - accuracy: 0.3204 - loss: 2.8519 - val_accuracy: 0.1399 - val_loss: 2.2040
Epoch 2/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 439s 1s/step - accuracy: 0.7738 - loss: 0.6551 - val_accuracy: 0.8071 - val_loss: 0.5455
Epoch 3/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 445s 1s/step - accuracy: 0.8393 - loss: 0.4674 - val_accuracy: 0.9016 - val_loss: 0.2879
Epoch 4/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 441s 1s/step - accuracy: 0.8674 - loss: 0.3871 - val_accuracy: 0.9275 - val_loss: 0.2094
Epoch 5/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 420s 1s/step - accuracy: 0.8916 - loss: 0.3099 - val_accuracy: 0.9208 - val_loss: 0.2316
Epoch 6/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 438s 1s/step - accuracy: 0.9006 - loss: 0.2830 - val_accuracy: 0.9072 - val_loss: 0.2586
Epoch 7/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 420s 1s/step - accuracy: 0.9049 - loss: 0.2756 - val_accuracy: 0.9247 - val_loss: 0.2144
Epoch 8/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 411s 1s/step - accuracy: 0.9173 - loss: 0.2273 - val_accu

NameError: name 'plt' is not defined

In [ ]:
print("\nClassification Report (Per-Command Accuracy):")
print(classification_report(y_test_labels, y_pred_labels, target_names=label_encoder.classes_))

# Calculate and plot confusion matrix
cm = confusion_matrix(y_test_labels, y_pred_labels, labels=label_encoder.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title('Confusion Matrix (DS-CNN Model)')
plt.show()

# Print the number of files executed
print(f"\nNumber of files executed for testing: {len(test_files)}")


Classification Report (Per-Command Accuracy):


NameError: name 'classification_report' is not defined

New_code
